In [40]:
import pandas as pd
import numpy as np
import pymssql
import mysql.connector

In [41]:
# 从sqlserver中获取数据
query = 'SELECT studentid, school, sex, age, famsize, medu AS mothereducation, fedu AS fathereducation, traveltime, studytime, failures, famrel, freetime, goout, g1 AS gradeperiod1, g2 AS gradeperiod2, g3 AS gradeperiod3 FROM studentmath'
server = "pdcc.c9sqqzd5fulv.us-west-2.rds.amazonaws.com"
user = "pdccuser"
password = "pdccpass"
database = "pdcctest"
conn = pymssql.connect(server=server, user=user, password=password, database=database)
studentmath = pd.read_sql(query, conn)
conn.close()

In [42]:
studentmath.dtypes

studentid          object
school             object
sex                object
age                 int64
famsize            object
mothereducation     int64
fathereducation     int64
traveltime          int64
studytime           int64
failures            int64
famrel              int64
freetime            int64
goout               int64
gradeperiod1        int64
gradeperiod2        int64
gradeperiod3        int64
dtype: object

In [43]:
studentmath.head()

,studentid,school,sex,age,famsize,mothereducation,fathereducation,traveltime,studytime,failures,famrel,freetime,goout,gradeperiod1,gradeperiod2,gradeperiod3
0,001,GP,F,18,GT3,4,4,2,2,0,4,3,4,5,6,6
1,002,GP,F,17,GT3,1,1,1,2,0,5,3,3,5,5,6
2,003,GP,F,15,LE3,1,1,1,2,3,4,3,2,7,8,10
3,004,GP,F,15,GT3,4,2,1,3,0,3,2,2,15,14,15
4,005,GP,F,16,GT3,3,3,1,2,0,4,3,2,6,10,10


默认打印5行

In [44]:
# 从mysql中获取数据
host = "pdccmysql.c9sqqzd5fulv.us-west-2.rds.amazonaws.com"
user = user
password = password
database = "pdccschema"
conn_mysql = mysql.connector.connect(host=host, user=user, password=password, database=database)
studentmath = pd.read_sql(query, conn_mysql)
conn_mysql.close()
studentmath.head()

,studentid,school,sex,age,famsize,mothereducation,fathereducation,traveltime,studytime,failures,famrel,freetime,goout,gradeperiod1,gradeperiod2,gradeperiod3
0,001,GP,F,18,GT3,4,4,2,2,0,4,3,4,5,6,6
1,002,GP,F,17,GT3,1,1,1,2,0,5,3,3,5,5,6
2,003,GP,F,15,LE3,1,1,1,2,3,4,3,2,7,8,10
3,004,GP,F,15,GT3,4,2,1,3,0,3,2,2,15,14,15
4,005,GP,F,16,GT3,3,3,1,2,0,4,3,2,6,10,10


In [46]:
# 只能运行一次，第二次时studentid已经被删除，再转化为index, studentmath[newColOrder]会引发keyError
newColOrder = ['studentid', 'gradeperiod1', 'gradeperiod2', 'gradeperiod3', 'school', 'sex', 'age', 'famsize', 'mothereducation', 'fathereducation', 'traveltime', 'studytime', 'freetime', 'failures', 'famrel', 'goout']
studentmath = studentmath[newColOrder]
# 确认studentid值唯一
studentmath.studentid.count()

395

In [47]:
studentmath.studentid.nunique()

395

In [48]:
# 设置Index, 只能运行一次
studentmath.set_index('studentid', inplace=True)

In [26]:
studentmath.columns

Index(['gradeperiod1', 'gradeperiod2', 'gradeperiod3', 'school', 'sex', 'age',
       'famsize', 'mothereducation', 'fathereducation', 'traveltime',
       'studytime', 'freetime', 'failures', 'famrel', 'goout'],
      dtype='object')

In [49]:
studentmath.count()

gradeperiod1       395
gradeperiod2       395
gradeperiod3       395
school             395
sex                395
age                395
famsize            395
mothereducation    395
fathereducation    395
traveltime         395
studytime          395
freetime           395
failures           395
famrel             395
goout              395
dtype: int64

In [50]:
# 替换成有意义的数据
setvalues={
    "famrel":{
        1:"1:very bad", 2:"2:bad", 3:"3:neutral", 4:"4:good", 5:"5:excellent"
    },
    "freetime":{
        1:"1:very low", 2:"2:low", 3:"3:neutral", 4:"4:high", 5:"5:very high"
    },
    "goout": {
        1:"1:very low", 2:"2:low", 3:"3:neutral", 4:"4:high", 5:"5:very high"
    },
    "mothereducation":{
        0:np.nan, 1:"1:k-4", 2:"2:5-9", 3:"3:secondary ed", 4:"4:higher ed"
    },
    "fathereducation":{
        0:np.nan, 1:"1:k-4", 2:"2:5-9", 3:"3:secondary ed", 4:"4:higher ed"
    }
}
studentmath.replace(setvalues, inplace=True)
setvalueskeys = [k for k in setvalues] # 字典中的键作为一个列表

In [51]:
# 查看内存使用情况
studentmath[setvalueskeys].memory_usage(index=False)

famrel             3160
freetime           3160
goout              3160
mothereducation    3160
fathereducation    3160
dtype: int64

In [52]:
for col in studentmath[setvalueskeys].columns:
    studentmath[col] = studentmath[col].astype('category')         # 上一单元中已修改的列，其类型改为category
# 再查看内存使用
studentmath[setvalueskeys].memory_usage(index=False)

famrel             595
freetime           595
goout              595
mothereducation    587
fathereducation    587
dtype: int64

改变数据类型后，内存使用量大大减少

In [53]:
# 所占百分比
studentmath['famrel'].value_counts(sort=False, normalize=True)

1:very bad     0.020253
2:bad          0.045570
3:neutral      0.172152
4:good         0.493671
5:excellent    0.268354
Name: famrel, dtype: float64

In [56]:
# 所占百分比
studentmath['famrel'].value_counts()

4:good         195
5:excellent    106
3:neutral       68
2:bad           18
1:very bad       8
Name: famrel, dtype: int64

In [57]:
# 所占百分比
studentmath['famrel'].value_counts(normalize=True)

4:good         0.493671
5:excellent    0.268354
3:neutral      0.172152
2:bad          0.045570
1:very bad     0.020253
Name: famrel, dtype: float64

In [54]:
# 多个列所占百分比
studentmath[['freetime', 'goout']].apply(pd.Series.value_counts, sort=False, normalize=True)

,freetime,goout
1:very low,0.048101,0.058228
2:low,0.162025,0.260759
3:neutral,0.397468,0.329114
4:high,0.291139,0.217722
5:very high,0.101266,0.134177
